<h1> Quant Agent Evaluations </h1>
The Quant Agent is a key component of our multi-agent system. It is designed to automate the extraction and structuring of financial information from SEC filings, such as 10-K reports.

For the evaluation of our Quant Agent, we test the accuracy of ** 17 financial **parameters across 10-K reports from various years across **8 companies**: Uber, Microsoft, Amazon, Adobe, Dell, Meta, Netflix, Nvidia

1. Installing all dependencies

In [ ]:
!pip install crewai
!pip install langchain_groq
!pip install crewai_tools
!pip install langchain_huggingface
!pip install langchain_huggingface

!pip install llama_index.llms.groq
!pip install llama-index-embeddings-huggingface
!pip install llama-parse
!pip install llama-index-llms-langchain
!pip install llama_index.evaluation
!pip install ragas

  Using cached tokenizers-0.20.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached tokenizers-0.20.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.51.3 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.20.3 which is incompatible.
  Using cached tokenizers-0.21.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached tokenizers-0.21.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Succe

**Dataset Setup** <br/>
We manually upload 10-K reports of these companies into this Collab. Ensure all files are uploaded to the /data directory. Alternatively, if you find these files hosted somewhere, you can also use !wget

In [ ]:
import os
os.makedirs('data', exist_ok=True)


# !wget "https://s23.q4cdn.com/407969754/files/doc_financials/2019/ar/Uber-Technologies-Inc-2019-Annual-Report.pdf" -O data/uber_10k.pdf
!wget "https://s2.q4cdn.com/470004039/files/doc_earnings/2023/q4/filing/_10-K-Q4-2023-As-Filed.pdf" -O data/apple_2023.pdf
# !wget "https://s2.q4cdn.com/470004039/files/doc_financials/2022/q4/_10-K-2022-(As-Filed).pdf" -O data/apple_2022.pdf
# !wget "https://s2.q4cdn.com/470004039/files/doc_financials/2021/q4/_10-K-2021-(As-Filed).pdf" -O data/apple_2021.pdf
# !wget "https://s2.q4cdn.com/470004039/files/doc_financials/2020/ar/_10-K-2020-(As-Filed).pdf" -O data/apple_2020.pdf
# !wget "https://www.dropbox.com/scl/fi/i6vk884ggtq382mu3whfz/apple_2019_10k.pdf?rlkey=eudxh3muxh7kop43ov4bgaj5i&dl=1" -O data/apple_2019.pdf

# download Tesla
!wget "https://ir.tesla.com/_flysystem/s3/sec/000162828024002390/tsla-20231231-gen.pdf" -O data/tesla_2023.pdf
# !wget "https://ir.tesla.com/_flysystem/s3/sec/000095017023001409/tsla-20221231-gen.pdf" -O data/tesla_2022.pdf
# !wget "https://www.dropbox.com/scl/fi/ptk83fmye7lqr7pz9r6dm/tesla_2021_10k.pdf?rlkey=24kxixeajbw9nru1sd6tg3bye&dl=1" -O data/amazon.pdf
# !wget "https://ir.tesla.com/_flysystem/s3/sec/000156459021004599/tsla-10k_20201231-gen.pdf" -O data/tesla_2020.pdf
# !wget "https://ir.tesla.com/_flysystem/s3/sec/000156459020004475/tsla-10k_20191231-gen_0.pdf" -O data/tesla_2019.pdf


Upload the excel file you want to fill with these parameters. The Structured LLM will take the financial metrics from the row 1 of this file!

In [ ]:
def parse_file(file_path: str) -> List[Document]:
    llama_parse = LlamaParse(
        api_key="<-Your LLamacloud API key ->",
        result_type='markdown',
    )

    result = llama_parse.load_data(
        file_path,
    )
    return result

In [ ]:
documents = parse_file('data/financial_metrics.xlsx')

Started parsing the file under job_id 78f76803-b6fe-4cf7-af53-ecc15c23b59e


We can see the 17 predefined financial metrics

In [ ]:
print(documents[0].text)

# in

|Year|Company|Accounts Receivable Turnover|Capital Expenditures|Current Ratio|Debt-to-Equity Ratio|EBITDA|Earnings Per Share (EPS)|Free Cash Flow|Gross Margin|Inventory Turnover|Net Income|Net Profit Margin|Operating Margin|Research and Development Expenses|Return on Assets (ROA)|Return on Equity (ROE)|Revenue|Year-over-Year Growth Rate|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|2024|adobe|NA|NA|NA|0.19|NA|NA|7.3|89|NA|5,560|26|31.3|3944|NA|NA|21505|12|



In [ ]:
text = documents[0].text

In [ ]:
import os
import json
import csv
from crewai import Agent, Task, Crew
from langchain_groq import ChatGroq

Define your Groq API Key below

In [ ]:
os.environ['GROQ_API_KEY'] = "<-ENTER-API-KEY-HERE->"

**Define all the LLMs required for evalutaion**

In [ ]:
RAG_llm = ChatGroq(
    model_name="groq/llama-3.3-70b-versatile",
    temperature=0.0,
    max_tokens=2000
)

Quant_llm = Groq(model="llama3-70b-8192", api_key="<-ENTER-API-KEY-HERE->")

Importing all the required libraries.

In [ ]:
from typing import List
from pydantic import BaseModel
import os
import json
import csv
from tqdm import tqdm
import time

from llama_index.core.llms import ChatMessage
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_parse import LlamaParse
from llama_index.llms.groq import Groq
from llama_index.core.schema import Document
from llama_index.core import Settings
import nest_asyncio

nest_asyncio.apply()

Define the data model for the Structured LLM to follow while generating an

In [ ]:
class CompanyParameters(BaseModel):

    Companies: List[str]
    FinancialParameters: List[str]
    Years: List[str]

Now we define our test companies.

***IMPORTANT*** <br/>
Due to Groq API limitations, we can only generate Evaluation outputs for 1 Company [sometimes 2] companies at a time, after which we see a time error. To circumvent this, we run the metric generation loop multiple times for different companies. Although the loop below might show a timeout error in the output, we have already run this code several times  and have obtained the output metrics.

In [ ]:
#test_companies = ["adobe", "dell", "microsoft","ibm", "meta", "netflix", "nvidia", "amazon"]
test_companies = [ "dell", "meta"]

In [ ]:
base_path = "data"

In [ ]:

# sllm = llm.as_structured_llm(output_cls=CompanyParameters)
# input_msg = ChatMessage.from_str(prompt)

The CrewAI loop to generate all the financial metrics from our RAG

In [ ]:
from tqdm import tqdm
import asyncio
async def generate_answers(company, financial_parameters: List[str], year) -> List[str]:
    companies_financial_parameters_answers = {}

    companies_financial_parameters_answers[year] = {}

    companies_financial_parameters_answers[year][company] = {}
    for financial_parameter in tqdm(financial_parameters):
        query = f"what is the {financial_parameter} of {company} for the year 2024?. Don't be verbose. Provide only 1 word for mathematical value nothing more. If you are unable to provide answer, output as NA."
        answer =  analytics_pipeline.kickoff(inputs={"question": query})
        companies_financial_parameters_answers[year][company][financial_parameter] = answer
        print(answer)
        time.sleep(45)
    return companies_financial_parameters_answers

In [ ]:
# answers =await generate_answers(companies, financial_parameters, years)

Helper function for flattening LLM response

In [ ]:
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)



Quant Metric Genertions!

In [ ]:
for company in test_companies:
    print("\n\n============================================================")
    print(f"Processing {company}...")

    report_path = f"{base_path}/{company}.pdf"

    inputs = {
        "question": f"What are the risk factors associated with {company}?"
    }
    prompt = f"""
    You are an AI assistant specializing in financial analysis. You've been given an Excel spreadsheet containing financial data for {company}. Your task is to extract and structure this information in a clear, organized format.

    The Excel sheet contains the following:
    1. Multiple companies (rows)
    2. Year (columns)
    3. Various financial parameters (sub-columns)

    Input Excel data:
    {text}

    Please present the extracted and structured information in a clear, easy-to-read format.
    """

    rag_tool = PDFSearchTool(
        pdf=report_path,
        config=dict(
            llm=dict(
                provider="groq",
                config=dict(
                    model="llama3-8b-8192",
                    temperature=0.0,
                ),
            ),
            embedder=dict(
                provider="huggingface",
                config=dict(
                    model="BAAI/bge-small-en-v1.5",
                    # task_type="retrieval_document",
                    # title="Embeddings",
                ),
            ),
        )
    )
    #rag_tool.query(f"what are the biggest risks of {company}?")
    RAG_Agent = Agent(
        role="Financial Data Extractor",
        goal="Use the information retrieved from the vectorstore to answer the question",
        backstory=(
            "You are an assistant for question-answering tasks. "
            "Use the information present in the retrieved context to answer the question. "
            "You have to provide a clear concise answer."
        ),
        llm=RAG_llm,
        verbose=True,
        allow_delegation=False,
    )

    RAG_task = Task(
        description="Extract information for the question {question} with the help of the respective tool.",
        expected_output="Return a clear and concise text as response.",
        agent=RAG_Agent,
        tools=[rag_tool],
    )

    sllm = llm.as_structured_llm(output_cls=CompanyParameters)

    input_msg = ChatMessage.from_str(prompt)

    output = sllm.chat([input_msg])

    output_obj = output.raw

    financial_parameters = output_obj.FinancialParameters

    years = output_obj.Years

    analytics_pipeline = Crew(
    agents=[RAG_Agent],
    tasks=[RAG_task],
    verbose=True,
    tpm_limit=1000,
    )
    answers =await generate_answers(company, financial_parameters, "2024")
    print(answers)





    flat_data = []
    for year, companies in answers.items():
        for company, metrics in companies.items():
            flat_metrics = flatten_dict(metrics)
            flat_metrics['Year'] = year
            flat_metrics['Company'] = company
            flat_data.append(flat_metrics)

    # Get all unique keys to use as CSV headers
    headers = set()
    for item in flat_data:
        headers.update(item.keys())

    # Sort headers to ensure 'Year' and 'Company' come first
    headers = sorted(headers)
    headers.insert(0, headers.pop(headers.index('Year')))
    headers.insert(1, headers.pop(headers.index('Company')))

    # Write to CSV
    with open('sec_10k_analysis_form_filling.csv', 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        writer.writeheader()
        for row in flat_data:
            writer.writerow(row)




Processing dell...


  0%|          | 0/17 [00:00<?, ?it/s]

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: bfa7e0c8-27df-4af8-b647-0cfcd75105ac                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 7a5a7471-437e-4668-b263-67bef85c659e
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 7a5a7471-437e-4668-b263-67bef85c659e
       Status: Executing Task...
    └── 🤖 Agent: Financial Data Extractor
            Status: In Progress

...

🚀 Crew: crew
└── 📋 Task: b024a403-5efe-4a6c-b8f7-d3a749b90f55
       Assigned to: Financial Data Extractor
       Status: ❌ Failed
    └── 🤖 Agent: Financial Data Extractor
            Status: In Progress
        └── ❌ LLM Failed

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: b024a403-5efe-4a6c-b8f7-d3a749b90f55                                                                     │
│  Agent: Financial Data Extractor                                                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: 6615f9a7-1c38-440c-865c-a986c2de0151                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

 82%|████████▏ | 14/17 [10:57<02:20, 46.99s/it]


RateLimitError: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01js3641zneg4a236bjh5ptd6p` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 98280, Requested 5369. Please try again in 52m32.444s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}


A sample output in terms of a Pandas dataframe

In [ ]:
import pandas as pd
from IPython.core.display import HTML

pd.set_option('display.max_colwidth', 10)

out_df = pd.read_csv("sec_10k_analysis_form_filling.csv")
html = out_df.to_html()
HTML(html)

,Year,Company,Accounts Receivable Turnover,Capital Expenditures,Current Ratio,Debt-to-Equity Ratio,Dividend Yield,EBITDA,Earnings Per Share (EPS),Free Cash Flow,Gross Margin,Inventory Turnover,Market Cap,Net Income,Net Profit Margin,Operating Margin,Price-to-Earnings (P/E) Ratio,Research and Development Expenses,Return on Assets (ROA),Return on Equity (ROE),Revenue,Year-over-Year Growth Rate
0,2019,Uber,NaN,2223,1.3,3.45,NaN,4.8B,($6.81),"$2,536,000",43.9,NaN,55.4B,-8.5 billion,NaN,0,NaN,4.5B,4.7%,NaN,14526,NaN


In [23]:
import pandas as pd
import json
import numpy as np
from typing import Dict, Any, Union, List, Tuple
import os.path

def load_data() -> Tuple[pd.DataFrame, Dict[str, Dict[str, Any]]]:
    try:
        excel_data = pd.read_excel('sec_10k_analysis_form_filling.xlsx')
    except Exception as e:
        print(f"Error loading Excel file: {e}")
        raise

    try:
        with open('companies.json', 'r') as f:
            json_data = json.load(f)
    except Exception as e:
        print(f"Error loading JSON file: {e}")
        raise

    return excel_data, json_data

def normalize_value(value: Any) -> Union[float, str, None]:
    if value is None or pd.isna(value) or value == "NA" or value == "Not specified" or value == "Not applicable":
        return None
    return value

def normalize_company_name(name: str) -> str:
    name = name.lower().strip()
    return name

def are_values_equal(val1: Any, val2: Any, tolerance: float = 0.01) -> bool:
    norm_val1 = normalize_value(val1)
    norm_val2 = normalize_value(val2)

    if norm_val1 is None and norm_val2 is None:
        return True

    if norm_val1 is None or norm_val2 is None:
        return False

    if isinstance(norm_val1, str) and isinstance(norm_val2, str):
        return norm_val1.lower() == norm_val2.lower()

    try:
        num1 = float(norm_val1)
        num2 = float(norm_val2)
        if abs(num1) > 100 or abs(num2) > 100:
            if abs(num1) < 1e-10 or abs(num2) < 1e-10:
                return abs(num1 - num2) < tolerance
            return abs((num1 - num2) / max(abs(num1), abs(num2))) < tolerance
        else:
            return abs(num1 - num2) < tolerance
    except (ValueError, TypeError):
        return False

def calculate_accuracy(excel_data: pd.DataFrame, json_data: Dict[str, Dict[str, Any]]) -> Dict[str, Any]:
    total_cells = 0
    correct_cells = 0
    company_accuracies = {}
    field_accuracies = {}
    error_details = []

    excel_columns = [col.lower() for col in excel_data.columns]

    for _, row in excel_data.iterrows():
        if 'company' in excel_columns:
            company_idx = excel_columns.index('company')
            company_name = normalize_company_name(str(row.iloc[company_idx]))
        else:
            print("No 'Company' column found in Excel file")
            continue

        company_key = None
        for key in json_data.keys():
            if normalize_company_name(key) == company_name:
                company_key = key
                break

        if company_key is None:
            print(f"Company {company_name} not found in JSON data")
            continue

        if 'year' in excel_columns:
            year_idx = excel_columns.index('year')
            excel_year = normalize_value(row.iloc[year_idx])
        else:
            excel_year = None

        json_year = normalize_value(json_data[company_key].get('Year'))

        if excel_year is not None and json_year is not None and excel_year != json_year:
            print(f"Year mismatch for {company_name}: Excel={excel_year}, JSON={json_year}")
            continue

        company_total = 0
        company_correct = 0

        for i, excel_col in enumerate(excel_columns):
            if excel_col.lower() in ['year', 'company']:
                continue

            json_field = None
            for field in json_data[company_key].keys():
                if field.lower() == excel_col:
                    json_field = field
                    break

            if json_field is None:
                continue

            excel_value = row.iloc[i]
            json_value = json_data[company_key][json_field]
            is_correct = are_values_equal(excel_value, json_value)

            total_cells += 1
            company_total += 1

            if is_correct:
                correct_cells += 1
                company_correct += 1
            else:
                error_details.append({
                    'Company': company_name,
                    'Field': excel_col,
                    'Excel Value': excel_value,
                    'JSON Value': json_value
                })

            if excel_col not in field_accuracies:
                field_accuracies[excel_col] = {'correct': 0, 'total': 0}
            field_accuracies[excel_col]['total'] += 1
            if is_correct:
                field_accuracies[excel_col]['correct'] += 1

        if company_total > 0:
            company_accuracies[company_name] = company_correct / company_total

    overall_accuracy = correct_cells / total_cells if total_cells > 0 else 0

    for field in field_accuracies:
        if field_accuracies[field]['total'] > 0:
            field_accuracies[field]['accuracy'] = field_accuracies[field]['correct'] / field_accuracies[field]['total']
        else:
            field_accuracies[field]['accuracy'] = 0

    return {
        'overall_accuracy': overall_accuracy,
        'company_accuracies': company_accuracies,
        'field_accuracies': {f: fa['accuracy'] for f, fa in field_accuracies.items()},
        'error_details': error_details,
        'total_cells': total_cells,
        'correct_cells': correct_cells
    }

def display_results(accuracy_results: Dict[str, Any]) -> None:
    print("\nACCURACY ANALYSIS RESULTS: \n")

    print(f"Overall Accuracy: {accuracy_results['overall_accuracy']:.2%}")
    print(f"Total Cells Evaluated: {accuracy_results['total_cells']}")
    print(f"Correct Cells: {accuracy_results['correct_cells']}")

    print("\nCompany-wise Accuracy\n")
    for company, accuracy in sorted(accuracy_results['company_accuracies'].items(), key=lambda x: x[1], reverse=True):
        print(f"{company}: {accuracy:.2%}")

    print("\nField-wise Accuracy\n")
    for field, accuracy in sorted(accuracy_results['field_accuracies'].items(), key=lambda x: x[1], reverse=True):
        print(f"{field}: {accuracy:.2%}")


def main():
    print("Starting accuracy analysis...")

    try:
        if not os.path.exists('sec_10k_analysis_form_filling.xlsx'):
            print("Error: Excel file 'sec_10k_analysis_form_filling.xlsx' not found")
            return

        if not os.path.exists('companies.json'):
            print("Error: JSON file 'companies.json' not found")
            return

        excel_data, json_data = load_data()
        accuracy_results = calculate_accuracy(excel_data, json_data)
        display_results(accuracy_results)
        error_df = pd.DataFrame(accuracy_results['error_details'])
        if not error_df.empty:
            error_df.to_csv('accuracy_errors.csv', index=False)


        summary = {
            'Overall Accuracy': [accuracy_results['overall_accuracy']],
            'Total Cells': [accuracy_results['total_cells']],
            'Correct Cells': [accuracy_results['correct_cells']]
        }
        pd.DataFrame(summary).to_csv('accuracy_summary.csv', index=False)

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()

Starting accuracy analysis...

ACCURACY ANALYSIS RESULTS: 

Overall Accuracy: 71.57%
Total Cells Evaluated: 102
Correct Cells: 73

Company-wise Accuracy

dell: 88.24%
adobe: 76.47%
meta: 70.59%
uber: 70.59%
amazon: 64.71%
microsoft: 58.82%

Field-wise Accuracy

current ratio: 100.00%
debt-to-equity ratio: 100.00%
ebitda: 100.00%
inventory turnover: 100.00%
capital expenditures: 83.33%
earnings per share (eps): 83.33%
accounts receivable turnover: 83.33%
free cash flow: 66.67%
gross margin: 66.67%
net income: 66.67%
net profit margin: 66.67%
operating margin: 66.67%
return on equity (roe): 66.67%
research and development expenses: 50.00%
return on assets (roa): 50.00%
revenue: 50.00%
year-over-year growth rate: 16.67%
